In [1]:
!pip install transformers
!pip install numpy==1.26.0
!pip install tensorflow[and-cuda]
!pip install torch torchvision torchaudio
!pip install --upgrade ipywidgets
!pip install tf-keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 23.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.19 requires numpy<2,>=1.26.4; python_version < "3.12", but you have numpy 1.26.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:0

In [2]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


## Data Preprocessing

## Baseline Model

In [9]:
import numpy as np
print(np.__version__)
from transformers import BertModel, BertTokenizer

1.26.0


In [10]:

# Load the BERT model and tokenizer
model = BertModel.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Process a sentence

input_text = "This is a sample sentence."

encoded_input = tokenizer(input_text, return_tensors="pt")
output = model(**encoded_input, output_hidden_states=True)

print(output.hidden_states[1].shape)

torch.Size([1, 8, 768])


In [11]:
%cd interpretablellm/data
! ls

[Errno 2] No such file or directory: 'interpretablellm/data'
/content/interpretablellm/data
runs  test_data.csv  train_data.csv


In [12]:
import torch.nn as nn
import torch
from transformers import AutoTokenizer, BertModel
import pandas as pd
class CustomBERTModel(nn.Module):
    def __init__(self, n):
          super(CustomBERTModel, self).__init__()
          self.bert = BertModel.from_pretrained("bert-base-uncased")
          ### New layers:
          self.flatten = nn.Flatten()
          self.linear1 = nn.Linear(768, 256)
          self.linear2 = nn.Linear(256, 1)
          self.linear3 = nn.Linear(512, 1)
          self.sigmoid = nn.Sigmoid()
          self.n = n

    def forward(self, ids, mask):
          embedding_output = self.bert.embeddings(input_ids)
          encoder_outputs = embedding_output
          for i, layer_module in enumerate(self.bert.encoder.layer):
              encoder_outputs = layer_module(encoder_outputs)[0]
              if i + 1 == self.n:
                  break

          linear1_output = self.linear1(encoder_outputs)
          print(linear1_output.shape)
          linear2_output = self.linear2(linear1_output)
          print(linear2_output.shape)
          flattened_output = self.flatten(linear2_output)
          print(flattened_output.shape)
          linear3_output = self.linear3(flattened_output)
          print(linear3_output.shape)
          sigmoid_output = self.sigmoid(linear3_output)
          print(sigmoid_output.shape)


          # sequence_output, pooled_output = self.bert(
          #      ids,
          #      attention_mask=mask)

          # # sequence_output has the following shape: (batch_size, sequence_length, 768)


          return sigmoid_output

# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# model = CustomBERTModel() # You can pass the parameters if required to have more flexible model
# model.to(torch.device("cpu")) ## can be gpu
# criterion = nn.CrossEntropyLoss() ## If required define your own criterion
# optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))

train = pd.read_csv("train_data.csv")

In [13]:
train['Sentiment'] = train['Sentiment'].replace({'Positive': 1, 'Negative': 0})
train

<ipython-input-13-d786a8a05c96>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['Sentiment'] = train['Sentiment'].replace({'Positive': 1, 'Negative': 0})


,Review,Sentiment
0,Updated: after first book arrived very damaged...,1
1,Missing the sketch pad. Even worse I realized ...,0
2,Every page has a crease running the entire len...,0
3,This was my 6th copy! All 5 of my kids had th...,1
4,Wonderful book but not crazy about using the w...,1
...,...,...
7995,Weak interpretation of Korean cooking. Not aut...,0
7996,If you want to read a book about a 6 month old...,0
7997,Activation codes DO NOT WORK. And Esri will no...,0
7998,Complete waste of time to read. Read Silent Sp...,0


In [14]:
outputs = []
preds = []
for idx, row in train.iterrows():
  try:
    input_text = row['Review']
    #encoded_input = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    #output = model(**encoded_input, output_hidden_states=True)
    # outputs.append(output.hidden_states[0])
    # outputs.append(encoded_input)
    outputs.append(input_text)
    preds.append(1 if row['Sentiment'] == 'Positive' else 0)
  except:
    continue
  if idx > 100:
    break

In [38]:
# max_len = max([out.shape[1] for out in outputs])  # Find the longest sequence

# # Pad all outputs to the same length
# padded_outputs = []
# for out in outputs:
#     padding_size = max_len - out.shape[1]
#     # Apply padding to the sequence along the second dimension (sequence_length)
#     padded_out = torch.nn.functional.pad(out, (0, 0, 0, padding_size))  # Pads the sequence length dimension
#     padded_outputs.append(padded_out)
# for i in range (len(padded_outputs)):
#   print(padded_outputs[i].shape)

torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([1, 261, 768])
torch.Size([

In [16]:
import torch
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label)
        }

dataset = TextDataset(outputs, preds, tokenizer, 512)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Iterate through the dataloader
for batch in dataloader:
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['label']

    # Use the data for training/evaluation
    print("Input IDs shape:", input_ids.shape)
    print("Attention Mask shape:", attention_mask.shape)
    print("Labels:", labels)

Input IDs shape: torch.Size([16, 512])
Attention Mask shape: torch.Size([16, 512])
Labels: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Input IDs shape: torch.Size([16, 512])
Attention Mask shape: torch.Size([16, 512])
Labels: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Input IDs shape: torch.Size([16, 512])
Attention Mask shape: torch.Size([16, 512])
Labels: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Input IDs shape: torch.Size([16, 512])
Attention Mask shape: torch.Size([16, 512])
Labels: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Input IDs shape: torch.Size([16, 512])
Attention Mask shape: torch.Size([16, 512])
Labels: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Input IDs shape: torch.Size([16, 512])
Attention Mask shape: torch.Size([16, 512])
Labels: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Input IDs shape: torch.Size([6, 512])
Attention Mask shape: torch.Size([6, 512])
Labels: tensor([0, 0, 0, 0, 0, 

In [39]:
# #X_tensor = torch.tensor(outputs)
# X_tensor = torch.cat(padded_outputs, dim=0)
# Y_tensor = torch.from_numpy(np.array(preds)).float()
# dataset = torch.utils.data.TensorDataset(X_tensor, Y_tensor)
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=True)

In [17]:
model_custom = CustomBERTModel(1) # You can pass the parameters if required to have more flexible model
model_custom.to(torch.device("cpu")) ## can be gpu
loss_fn = nn.BCELoss() ## If required define your own criterion
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model_custom.parameters()))

In [18]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(dataloader):
        # Every data instance is an input + label pair
        print(data)
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(dataloader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

In [19]:
#tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-italian-xxl-cased")
model = CustomBERTModel(1) # You can pass the parameters if required to have more flexible model
model.to(torch.device("cpu")) ## can be gpu
criterion = nn.BCELoss() ## If required define your own criterion
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))

epochs = 1
for epoch in range(epochs):
    for batch in dataloader: ## If you have a DataLoader()  object to get the data.
        print(batch)
        #encoding = batch[0]
        targets = np.array(batch['label']) ## assuming that data loader returns a tuple of data and its targets
        targets = torch.tensor(np.expand_dims(targets,axis=1)).float()
        print(targets.shape)
        optimizer.zero_grad()
        #encoding = tokenizer.batch_encode_plus(data, return_tensors='pt', padding=True, truncation=True,max_length=50, add_special_tokens = True)
        input_ids = batch['input_ids']
        #print(input_ids.shape)
        attention_mask = batch['attention_mask']
        #print(attention_mask.shape)
        outputs = model(input_ids, attention_mask)
        #outputs = F.log_softmax(outputs, dim=1)
        #print(outputs.shape)
        loss = criterion(outputs, targets)
        print(loss)
        loss.backward()
        optimizer.step()

{'input_ids': tensor([[  101,  2023,  1037,  ...,     0,     0,     0],
        [  101,  1997,  2607,  ...,     0,     0,     0],
        [  101,  2296,  3931,  ...,     0,     0,     0],
        ...,
        [  101, 10140,  2338,  ...,     0,     0,     0],
        [  101,  2293,  2035,  ...,     0,     0,     0],
        [  101,  1045,  5959,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'label': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}
torch.Size([16, 1])
torch.Size([16, 512, 256])
torch.Size([16, 512, 1])
torch.Size([16, 512])
torch.Size([16, 1])
torch.Size([16, 1])
tensor(0.6831, grad_fn=<BinaryCrossEntropyBackward0>)
{'input_ids': tensor([[  101,  1045,  2031,  ...,     0,     0,     0],
        [  101,  2307,  4979,  ...,     0,     0,     0],
 

In [20]:
print(loss)

tensor(0.0001, grad_fn=<BinaryCrossEntropyBackward0>)


In [7]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')
unmasker("The man worked as a [MASK].")

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

[{'score': 0.09747565537691116,
  'token': 10533,
  'token_str': 'carpenter',
  'sequence': 'the man worked as a carpenter.'},
 {'score': 0.052383214235305786,
  'token': 15610,
  'token_str': 'waiter',
  'sequence': 'the man worked as a waiter.'},
 {'score': 0.04962708428502083,
  'token': 13362,
  'token_str': 'barber',
  'sequence': 'the man worked as a barber.'},
 {'score': 0.037886086851358414,
  'token': 15893,
  'token_str': 'mechanic',
  'sequence': 'the man worked as a mechanic.'},
 {'score': 0.037680841982364655,
  'token': 18968,
  'token_str': 'salesman',
  'sequence': 'the man worked as a salesman.'}]

In [9]:
! git clone https://github.com/avyas21/interpretablellm.git

fatal: destination path 'interpretablellm' already exists and is not an empty directory.


In [26]:
def get_review(review):
  prompt = "Return whether the sentiment is positive or negative."
  unmasker = pipeline('fill-mask', model='bert-base-uncased')
  return unmasker("Review: " + review + "This review was [MASK]")

In [27]:
get_review("A well thought out book that teaches my kids manners and how to be polite!!! Could it get any better?!? There are 30 beautiful designed and illustrated pages in the continuing story of My Travel Friends. This series just gets better and better! The interactive app really brings the book to life with voices and songs!")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


[{'score': 0.4765911102294922,
  'token': 1012,
  'token_str': '.',
  'sequence': 'review : a well thought out book that teaches my kids manners and how to be polite!!! could it get any better?!? there are 30 beautiful designed and illustrated pages in the continuing story of my travel friends. this series just gets better and better! the interactive app really brings the book to life with voices and songs! this review was.'},
 {'score': 0.2635924816131592,
  'token': 1025,
  'token_str': ';',
  'sequence': 'review : a well thought out book that teaches my kids manners and how to be polite!!! could it get any better?!? there are 30 beautiful designed and illustrated pages in the continuing story of my travel friends. this series just gets better and better! the interactive app really brings the book to life with voices and songs! this review was ;'},
 {'score': 0.1289989948272705,
  'token': 1029,
  'token_str': '?',
  'sequence': 'review : a well thought out book that teaches my kids 

In [ ]:
model = BertModel